In [1]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

In [2]:
# Loading the datasets
movies_data = pd.read_csv('/content/movies.csv')

Exploratory Data Analysis

In [3]:
movies_data.head()

,index,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,cast,crew,director
0,0,237000000,Action Adventure Fantasy Science Fiction,http://www.avatarmovie.com/,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,Sam Worthington Zoe Saldana Sigourney Weaver S...,"[{'name': 'Stephen E. Rivkin', 'gender': 0, 'd...",James Cameron
1,1,300000000,Adventure Fantasy Action,http://disney.go.com/disneypictures/pirates/,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082615,...,169.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"At the end of the world, the adventure begins.",Pirates of the Caribbean: At World's End,6.9,4500,Johnny Depp Orlando Bloom Keira Knightley Stel...,"[{'name': 'Dariusz Wolski', 'gender': 2, 'depa...",Gore Verbinski
2,2,245000000,Action Adventure Crime,http://www.sonypictures.com/movies/spectre/,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bond’s past sends him o...,107.376788,...,148.0,"[{""iso_639_1"": ""fr"", ""name"": ""Fran\u00e7ais""},...",Released,A Plan No One Escapes,Spectre,6.3,4466,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,"[{'name': 'Thomas Newman', 'gender': 2, 'depar...",Sam Mendes
3,3,250000000,Action Crime Drama Thriller,http://www.thedarkknightrises.com/,49026,dc comics crime fighter terrorist secret ident...,en,The Dark Knight Rises,Following the death of District Attorney Harve...,112.312950,...,165.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,The Legend Ends,The Dark Knight Rises,7.6,9106,Christian Bale Michael Caine Gary Oldman Anne ...,"[{'name': 'Hans Zimmer', 'gender': 2, 'departm...",Christopher Nolan
4,4,260000000,Action Adventure Science Fiction,http://movies.disney.com/john-carter,49529,based on novel mars medallion space travel pri...,en,John Carter,"John Carter is a war-weary, former military ca...",43.926995,...,132.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}]",Released,"Lost in our world, found in another.",John Carter,6.1,2124,Taylor Kitsch Lynn Collins Samantha Morton Wil...,"[{'name': 'Andrew Stanton', 'gender': 2, 'depa...",Andrew Stanton


**About the Features**
index: A unique identifier for each movie entry in the dataset.
Example: 0 (for "3 Idiots", assuming it is the first movie in the dataset)

budget: The amount of money spent on producing the movie.
Example: 55000000 (for 3 Idiots, let's assume INR 55 crores, converted to USD)

genres: The categories or types of the movie.
Example: ['Comedy', 'Drama'] (since "3 Idiots" is a mix of both genres)

homepage: The official website of the movie.
Example: https://www.3idiots.in (fictional example for illustration)

id: A unique identifier (like TMDB or IMDb ID) for each movie.
Example: tt1187043 (the IMDb ID for "3 Idiots")

keywords: Important words or phrases associated with the movie.
Example: ['friendship', 'college', 'inspiration', 'education', 'engineering']

original_language: The language in which the movie was originally made.
Example: Hindi (since "3 Idiots" is originally in Hindi)

original_title: The movie's original title.
Example: 3 Idiots (as it is known worldwide by the same name)

overview: A short description or summary of the movie.
Example: "Two friends search for their long-lost college buddy, who inspired them to think differently, while dealing with societal pressures and academic challenges."

popularity: A score indicating how popular the movie is (calculated using views, ratings, etc.).
Example: 95.6 (this is a made-up number representing its popularity score)

production_companies: The production studios or companies involved in making the film.
Example: ['Vidhu Vinod Chopra Productions']

production_countries: The countries where the film was produced.
Example: ['India'] (since "3 Idiots" was produced in India)

release_date: The date when the movie was released.
Example: 2009-12-25 (for "3 Idiots", it was released on Christmas Day 2009)

revenue: The total earnings or box office collection of the movie.
Example: 2.2E8 (assuming it earned INR 220 crores, converted to USD)

runtime: The duration of the movie in minutes.
Example: 170 (since "3 Idiots" runs for 170 minutes)

spoken_languages: The languages spoken in the movie.
Example: ['Hindi', 'English'] (some dialogues might be in English as well)

status: The current status of the movie (e.g., Released, Post-production, etc.).
Example: Released (since "3 Idiots" is already released)

tagline: A catchy phrase used to promote the movie.
Example: "Don't be Stupid, Be an Idiot!" (fictional tagline for illustration)

title: The official name of the movie.
Example: 3 Idiots (this is the official title)

vote_average: The average rating the movie has received from viewers.
Example: 8.4 (since 3 Idiots has a high IMDb rating)

vote_count: The number of votes or ratings the movie has received.
Example: 560,000 (approximate number of IMDb votes for 3 Idiots)

cast: The main actors or characters in the movie.
Example: ['Aamir Khan', 'R. Madhavan', 'Sharman Joshi', 'Kareena Kapoor']

crew: The people involved in making the film (e.g., writers, cinematographers, music directors, etc.).
Example: ['Rajkumar Hirani (Director)', 'Abhijat Joshi (Writer)', 'Shantanu Moitra (Music)']

director: The name of the person who directed the movie.
Example: Rajkumar Hirani (he directed "3 Idiots")



In [5]:
movies_data.shape

(4803, 24)

In [4]:
movies_data.columns

Index(['index', 'budget', 'genres', 'homepage', 'id', 'keywords',
       'original_language', 'original_title', 'overview', 'popularity',
       'production_companies', 'production_countries', 'release_date',
       'revenue', 'runtime', 'spoken_languages', 'status', 'tagline', 'title',
       'vote_average', 'vote_count', 'cast', 'crew', 'director'],
      dtype='object')

In [7]:
# Checking for the null values count in the dataset
movies_data.isnull().sum()

,0
index,0
budget,0
genres,28
homepage,3091
id,0
keywords,412
original_language,0
original_title,0
overview,3
popularity,0


In [8]:
# selecting the relevant features for recommendation
selected_features = ['genres','keywords','tagline','cast','director']
print(selected_features)

['genres', 'keywords', 'tagline', 'cast', 'director']


In [10]:
# Managing the Null Values
for feature in selected_features:
  movies_data[feature]=movies_data[feature].fillna('')

In [11]:
# combining the selected features
combined_features = movies_data['genres']+movies_data['keywords']+movies_data['tagline']+movies_data['cast']+movies_data['director']

In [12]:
# converting the text data to feature vectors
vectorizer = TfidfVectorizer()
feature_vectors = vectorizer.fit_transform(combined_features)

In [13]:
print(feature_vectors)

  (0, 245)	0.07858581760804864
  (0, 444)	0.09109355212252296
  (0, 8936)	0.11806131645084655
  (0, 21836)	0.09966592997173947
  (0, 9304)	0.27385789694599244
  (0, 4456)	0.2179967521551067
  (0, 10193)	0.1653243242081299
  (0, 22916)	0.3366875640692919
  (0, 26334)	0.130597230719163
  (0, 4676)	0.24529097704249617
  (0, 22778)	0.27385789694599244
  (0, 24197)	0.07518543993419267
  (0, 27161)	0.12619886579371511
  (0, 18164)	0.08690831799482268
  (0, 18558)	0.26121683351048536
  (0, 21521)	0.15623566659433683
  (0, 27182)	0.23480088356130557
  (0, 27540)	0.19771357974524179
  (0, 21487)	0.21550128478931552
  (0, 22464)	0.2020708956871175
  (0, 26547)	0.19638671147741735
  (0, 23267)	0.16128139780622522
  (0, 14550)	0.22368093636480682
  (0, 16614)	0.1569607387745327
  (0, 20729)	0.27385789694599244
  :	:
  (4801, 18790)	0.22450140828104795
  (4801, 11776)	0.3005604008026175
  (4801, 758)	0.19369913462690624
  (4801, 8121)	0.2611505149930138
  (4801, 27513)	0.3005604008026175
  (4801, 2

Cosine similarity

In [14]:
# determing the similarity scores
similarity = cosine_similarity(feature_vectors)
print(similarity)

[[1.         0.06865296 0.01492221 ... 0.         0.         0.        ]
 [0.06865296 1.         0.02799128 ... 0.01243107 0.         0.        ]
 [0.01492221 0.02799128 1.         ... 0.         0.         0.        ]
 ...
 [0.         0.01243107 0.         ... 1.         0.         0.        ]
 [0.         0.         0.         ... 0.         1.         0.        ]
 [0.         0.         0.         ... 0.         0.         1.        ]]


In [15]:
print(similarity.shape)

(4803, 4803)


Getting the movies name from the user

In [18]:
movie_name = input('Enter your Favourite Movie Name: ')

Enter your Favourite Movie Name: Avator


In [17]:
# checking the name of movies exits in the dataset
list_of_all_titles = movies_data['title'].tolist()
print(list_of_all_titles)

['Avatar', "Pirates of the Caribbean: At World's End", 'Spectre', 'The Dark Knight Rises', 'John Carter', 'Spider-Man 3', 'Tangled', 'Avengers: Age of Ultron', 'Harry Potter and the Half-Blood Prince', 'Batman v Superman: Dawn of Justice', 'Superman Returns', 'Quantum of Solace', "Pirates of the Caribbean: Dead Man's Chest", 'The Lone Ranger', 'Man of Steel', 'The Chronicles of Narnia: Prince Caspian', 'The Avengers', 'Pirates of the Caribbean: On Stranger Tides', 'Men in Black 3', 'The Hobbit: The Battle of the Five Armies', 'The Amazing Spider-Man', 'Robin Hood', 'The Hobbit: The Desolation of Smaug', 'The Golden Compass', 'King Kong', 'Titanic', 'Captain America: Civil War', 'Battleship', 'Jurassic World', 'Skyfall', 'Spider-Man 2', 'Iron Man 3', 'Alice in Wonderland', 'X-Men: The Last Stand', 'Monsters University', 'Transformers: Revenge of the Fallen', 'Transformers: Age of Extinction', 'Oz: The Great and Powerful', 'The Amazing Spider-Man 2', 'TRON: Legacy', 'Cars 2', 'Green Lant

In [19]:
# determining the close match of the movie given by the user
find_close_match = difflib.get_close_matches(movie_name,list_of_all_titles)
print(find_close_match)

['Avatar', 'The Aviator', 'Anatomy']


In [20]:
# finding the index of the movie
index_of_the_movie=movies_data[movies_data.title==find_close_match[0]]['index'].values[0]
print(index_of_the_movie)

0


In [25]:
# similarity of movies
similarity_score = list(enumerate(similarity[index_of_the_movie]))
print(similarity_score)
print("Similarity Score :",len(similarity_score))

[(0, 0.9999999999999999), (1, 0.06865295547859393), (2, 0.014922211095123292), (3, 0.006767505107039975), (4, 0.09507382234152614), (5, 0.028425087268809008), (6, 0.008543064750892471), (7, 0.024556550804070994), (8, 0.03142328987292023), (9, 0.01460905165226577), (10, 0.06923400831178866), (11, 0.013432681129643647), (12, 0.02268441155914605), (13, 0.019500508535135064), (14, 0.060954783968025214), (15, 0.00803832886848643), (16, 0.015430503883444694), (17, 0.014243100737474104), (18, 0.021175287042263805), (19, 0.02923831093363255), (20, 0.021311225708366407), (21, 0.005517022982915349), (22, 0.020579978079258272), (23, 0.037978244359993626), (24, 0.035153578653387814), (25, 0.029294071728024874), (26, 0.06262210806872326), (27, 0.02207837838293123), (28, 0.02376479356993962), (29, 0.015335880492985836), (30, 0.029600891245345873), (31, 0.052467706091446034), (32, 0.025733700953020428), (33, 0.02478190925239012), (34, 0.0), (35, 0.017235500233347474), (36, 0.03260955224712312), (37, 

In [26]:
# sorting the movies based on their similarity score

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)
print(sorted_similar_movies)

[(0, 0.9999999999999999), (3158, 0.2369597611206783), (94, 0.2053455297464646), (1531, 0.161534730660534), (1951, 0.1543083447512902), (838, 0.15204587212274273), (2403, 0.14740843210123528), (206, 0.14702326102726826), (1759, 0.14106195056024493), (56, 0.1336598621652937), (1473, 0.12495339086853643), (1053, 0.12200369591270702), (1650, 0.11973442360362235), (47, 0.11925702605517305), (43, 0.11806680574459066), (3730, 0.11791721007846388), (278, 0.11692949605440733), (3105, 0.1160317914332425), (1099, 0.1153110863686499), (942, 0.11434953802438658), (1275, 0.1138100347457997), (812, 0.11147618168839672), (2060, 0.10917237745379887), (1804, 0.10642553392218385), (3056, 0.1036520554952102), (977, 0.10351595902470023), (305, 0.10238849811317507), (3208, 0.10223343084498923), (158, 0.10204495975306646), (150, 0.10144262602097147), (2827, 0.10105423029707622), (3439, 0.09881402857772717), (342, 0.09840693019612001), (775, 0.09723826197147899), (1922, 0.09515295396425277), (4, 0.09507382234

In [27]:
# print the name of similar movies based on the index
print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

Movies suggested for you : 

1 . Avatar
2 . Alien
3 . Guardians of the Galaxy
4 . Moonraker
5 . Space Dogs
6 . Alien³
7 . Aliens
8 . Clash of the Titans
9 . The Right Stuff
10 . Star Trek Beyond
11 . The Astronaut's Wife
12 . Galaxy Quest
13 . Wing Commander
14 . Star Trek Into Darkness
15 . Terminator Salvation
16 . Cargo
17 . Planet of the Apes
18 . Imaginary Heroes
19 . Shadow Conspiracy
20 . The Book of Life
21 . Sunshine
22 . Pocahontas
23 . Out of the Furnace
24 . Snow White: A Tale of Terror
25 . Cedar Rapids
26 . The Iron Giant
27 . Treasure Planet
28 . Star Wars: Clone Wars: Volume 1
29 . Star Trek


**MOVIE RECOMMENDATION SYSTEM**

In [28]:
movie_name = input(' Enter your favourite movie name : ')

list_of_all_titles = movies_data['title'].tolist()

find_close_match = difflib.get_close_matches(movie_name, list_of_all_titles)

close_match = find_close_match[0]

index_of_the_movie = movies_data[movies_data.title == close_match]['index'].values[0]

similarity_score = list(enumerate(similarity[index_of_the_movie]))

sorted_similar_movies = sorted(similarity_score, key = lambda x:x[1], reverse = True)

print('Movies suggested for you : \n')
i = 1
for movie in sorted_similar_movies:
  index = movie[0]
  title_from_index = movies_data[movies_data.index==index]['title'].values[0]
  if (i<30):
    print(i, '.',title_from_index)
    i+=1

 Enter your favourite movie name : Iron MAn
Movies suggested for you : 

1 . Iron Man
2 . Iron Man 2
3 . Iron Man 3
4 . The Avengers
5 . Captain America: Civil War
6 . The Helix... Loaded
7 . Avengers: Age of Ultron
8 . X-Men: The Last Stand
9 . X-Men: Days of Future Past
10 . X-Men
11 . X2
12 . Made
13 . Tropic Thunder
14 . The Incredible Hulk
15 . X-Men: Apocalypse
16 . The Amazing Spider-Man 2
17 . Captain America: The Winter Soldier
18 . Gothika
19 . X-Men: First Class
20 . A Scanner Darkly
21 . The Judge
22 . Deadpool
23 . The Best Man
24 . The Nativity Story
25 . Lucky You
26 . The Kite Runner
27 . Arlington Road
28 . Ant-Man
29 . The Hunting Party
